In [43]:
import os
import tqdm
import torch
import numpy as np
import open3d as o3d

import matplotlib
import matplotlib.pyplot as plt

import sys
sys.path.append(os.path.abspath(".."))
from lib.dataloader import kitti_loader
from lib.dataloader.kitti_loader import load_lidar_point
from lib.utils.image_utils import color_mapping


def vis_pcd(points, point_size=0.3, colormap=matplotlib.colormaps["rainbow"]):
   
    # 计算点云重心
    centroid = np.mean(points, axis=0)
    
    # 将点云移动到重心位置
    points = points - centroid
    
    pcd = o3d.geometry.PointCloud()
    # 更新点云数据
    pcd.points = o3d.utility.Vector3dVector(points)

    z = points[:, 2]
    max_q = np.quantile(z, 0.9)
    min_q = np.quantile(z, 0.1)
    z = (z - min_q) / (max_q - min_q)
    z = np.clip(z, 0, 1)

    color = color_mapping(z, colormap)
    pcd.colors = o3d.utility.Vector3dVector(color)

    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(pcd)
    
    render_option = vis.get_render_option()
    render_option.point_size = point_size
    render_option.background_color = np.array([1.0, 1.0, 1.0])

    vis.run()
    vis.destroy_window()

In [ ]:
class Args:
    def __init__(self):
        self.data_dir = "../data/kitti360"
        self.data_type = "KITTI"
        self.frame_length = [1908, 1971]
        self.eval_frames = [1921, 1934, 1947, 1960]
        self.scene_id = "ks3"
        self.dynamic = False

args = Args()

seq = "0000"
full_seq = f"2013_05_28_drive_{seq}_sync"
lidar_points = load_lidar_point(
    os.path.join(args.data_dir, "data_3d_raw", full_seq, "velodyne_points", "data"),
    args.frame_length,
)

lidar, bbox = kitti_loader.load_kitti_raw(args.data_dir, args)

all_points = []
for frame in range(args.frame_length[0], args.frame_length[1] + 1):
    points = lidar_points[frame]
    all_points.append(points)
all_points = np.concatenate(all_points, axis=0)

In [ ]:
all_points_world = []
for frame in range(args.frame_length[0], args.frame_length[1] + 1):
    points, intensity = lidar.inverse_projection(frame)
    points_world = (lidar.sensor2world[1908] @ torch.cat([points, torch.ones_like(points[:, :1])], dim=-1).T).T
    all_points_world.append(points_world)
all_points_world = np.concatenate(all_points_world, axis=0)
vis_pcd(all_points_world, point_size=0.5)


In [ ]:
frame = 1938
points, intensity = lidar.inverse_projection(frame)
# vis_pcd(points.detach().cpu().numpy(), point_size=1.)
points.shape

In [ ]:
135960 / 32